In [1]:
#! pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_file = requests.get(url).text
soup =BeautifulSoup(html_file,'lxml')


In [2]:
import pandas as pd
import numpy as np
match = soup.title.text  #access like an attribute
context=soup.find('div', class_='mw-parser-output')  #find can indicate what tag you wanna find

    

In [3]:
data =[
    
        [[td.text.strip() for td in tr.find_all('td') ]
        for tr in table.find_all('tr')[1:]] 
    for table in soup.find_all('table')]

data2=data[0]


In [4]:
df=pd.DataFrame(data2, columns=['Postal Code', 'Borough', 'Neighborhood'])

In [5]:
df.describe()

,Postal Code,Borough,Neighborhood
count,289,289,289
unique,180,12,210
top,M9V,Not assigned,Not assigned
freq,8,77,78


In [6]:
df = df[df.Borough !='Not assigned']  #delete rows with borough 'not assigned'
df.reset_index(inplace=True)
df = df[['Postal Code','Borough','Neighborhood']]
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
df.index[df['Neighborhood']=='Not assigned'].tolist()

[6]

In [8]:
df=df.replace(to_replace=df.iloc[6,2],value=df.iloc[6,1],regex=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
df=df.groupby('Postal Code', as_index=False).agg(lambda x:','.join(set(x.dropna())))

In [10]:
df.shape

(103, 3)

In [11]:
dfl=pd.read_csv('Geospatial_Coordinates.csv')
dfl.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
result = pd.merge(df, dfl, on='Postal Code', how='outer')

result.columns = ['Postalcode', 'Borough','Neighborhood','Latitude','Longitude']
result.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
#count the number of postal code each borough
df.groupby('Borough').count()

,Postal Code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,18,18
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Queen's Park,1,1
Scarborough,17,17


In [14]:
#generate map of the neighborhood
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following packages will be UPDATED:

  ca-certificates                     2018.11.29-ha4d7672_0 --> 2019.3.9-hecc5488_0
  certifi                              2018.11.29-py36_1000 --> 2019.3.9-py36_0
  conda                                        4.6.4-py36_0 --> 4.6

In [15]:
# create map and display the different zipcodes.

Tor_map = folium.Map(location=[43.8, -79.2], zoom_start=10)

for lat, lng, PC in zip(result.Latitude, result.Longitude, result.Postalcode ):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        fill=True,
        popup=PC,
        fill_color='blue',
        fill_opacity=0.6
       ).add_to(Tor_map)
# display the map 
Tor_map